# Description du projet : 

Contexte :  L'INSEE est l'institut officiel français qui collecte des données de tous types sur le territoire français. Elles peuvent être démographiques (Naissances, Décès, Densité de la population...), économiques (Salaires, Entreprises par activité / taille...) et plus encore.
Ces données peuvent être d'une grande aide pour observer et mesurer les inégalités au sein de la population française.

## Objectif : Comparer les inégalités en France : 

* Entreprises en fonction de leur localisation, de leur taille. 
* Population en fonction du salaire et de la localisation.
* Focus sur une grande ville 


In [1]:
# Suppression des warnings
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

## Données géographiques sur les villes françaises (principalement la latitude et la longitude, mais aussi les codes et les noms des régions/départements).

<u>geo_2020.csv : </u> Fichier de synthèse de Dominique avec les coordonnées GPS

* COM (équivalent CODGEO)
* Code_postal
* latitude
* longitude

<u>communes2020.csv : </u> Synthèse des données issues de l'INSEE

* typecom	
* com
* reg
* libelle 


In [2]:
# Utilisation du fichier données GPS (étude Dominique)
geo = pd.read_csv('geo_2020.csv',dtype={'Code_postal':'str'}
                  ,usecols=['COM','LIBELLE_DEP','LIBELLE_REG','Code_postal','latitude','longitude'])
# Suppression des doublons
geo = geo.drop_duplicates(subset='COM',keep='first')

def complete_cp(cp):
    if len(str(cp))==4:
        cp = '0' + cp
    return str(cp)

geo['Code_postal'] = geo['Code_postal'].apply(lambda x: complete_cp(x))

# Affichage des premières lignes
geo.head()

,COM,LIBELLE_DEP,LIBELLE_REG,Code_postal,latitude,longitude
0,01001,Ain,Auvergne-Rhône-Alpes,01400,46.153721,4.925850
1,01002,Ain,Auvergne-Rhône-Alpes,01640,46.009606,5.428088
2,01004,Ain,Auvergne-Rhône-Alpes,01500,45.961049,5.372275
3,01005,Ain,Auvergne-Rhône-Alpes,01330,45.996164,4.911967
4,01006,Ain,Auvergne-Rhône-Alpes,01300,45.749886,5.594585


In [3]:
# Mise jour des nan pour Paris
geo.loc[geo.loc[geo.COM=='75056'].index,'longitude'] = np.float64(geo.loc[geo.loc[geo.COM=='75101'].index,'longitude'].values)
geo.loc[geo.loc[geo.COM=='75056'].index,'latitude'] = np.float64(geo.loc[geo.loc[geo.COM=='75101'].index,'latitude'].values)
geo.loc[geo.loc[geo.COM=='75056'].index,'Code_postal'] = '75000'

In [4]:
geo.shape

(37026, 6)

In [5]:
df2_2020 = pd.read_csv('communes2020.csv',dtype='str',usecols=['typecom','com','reg','dep','libelle'])
# Affichage des infos
df2_2020.info() 
# Affichage des premières lignes
df2_2020.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37901 entries, 0 to 37900
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   typecom  37901 non-null  object
 1   com      37901 non-null  object
 2   reg      35013 non-null  object
 3   dep      35013 non-null  object
 4   libelle  37901 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


,typecom,com,reg,dep,libelle
0,COM,01001,84,01,L'Abergement-Clémenciat
1,COM,01002,84,01,L'Abergement-de-Varey
2,COM,01004,84,01,Ambérieu-en-Bugey
3,COM,01005,84,01,Ambérieux-en-Dombes
4,COM,01006,84,01,Ambléon


In [6]:
# Suppression des communes déléguées (non présent dans le dataset entreprises et salaires)
index_to_drop = df2_2020.loc[(df2_2020.typecom=='COMD')| (df2_2020.typecom=='COMA')].index
df2_2020 = df2_2020.drop(index_to_drop)

# # modification du numero de commune pour coherence avec les autres fichiers
index_temp = df2_2020.loc[df2_2020.com=='27676'].index
df2_2020.loc[index_temp,'com'] = '27058'
df2_2020.loc[df2_2020.com=='27058']


,typecom,com,reg,dep,libelle
10760,COM,27058,28,27,Les Trois Lacs


In [7]:
df2_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35013 entries, 0 to 37900
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   typecom  35013 non-null  object
 1   com      35013 non-null  object
 2   reg      35013 non-null  object
 3   dep      35013 non-null  object
 4   libelle  35013 non-null  object
dtypes: object(5)
memory usage: 2.6+ MB


In [8]:
# Ajout de EU_circo, libelle région et chef lieu (source regions.csv)
dict_EU_circo = {'code_région': ['01','02','03','04','06','11','24','27','28','32','44','52','53','75','76','84','93','94'],
                 'EU_circo':['Outre-Mer','Outre-Mer','Outre-Mer','Outre-Mer','Outre-Mer','Île-de-France',
                             'Centre','Est','Nord-Ouest','Nord-Ouest','Est','Ouest','Ouest','Sud-Ouest',
                             'Sud-Ouest','Sud-Est','Sud-Est','Sud-Est'],
                 'code_chef_lieu' : ['97105','97209','97302','97411','97608','75056','45234','21231','76540',
                                       '59350','67482','44109','35238','33063','31555','69123','13055','2A004']}

df_eu_circo = pd.DataFrame(dict_EU_circo)
list_eu_circo = []
for i in df_eu_circo.code_chef_lieu:
    liste_temp = df2_2020.loc[df2_2020.com==i].libelle.values
    list_eu_circo = np.append(list_eu_circo,liste_temp)

df_eu_circo['chef_lieu_libelle'] = pd.Series(list_eu_circo)

def comp_eu_circo (reg):
    eu_circo = df_eu_circo.loc[df_eu_circo.code_région==reg].EU_circo.values[0]
    return eu_circo

def comp_code_cl (reg):
    code_chef_lieu = df_eu_circo.loc[df_eu_circo.code_région==reg].code_chef_lieu.values[0]
    return code_chef_lieu

def comp_cl_libelle (reg):
    chef_lieu_libelle = df_eu_circo.loc[df_eu_circo.code_région==reg].chef_lieu_libelle.values[0]
    return chef_lieu_libelle

df2_2020['EU_circo'] = df2_2020.reg.apply(lambda x: comp_eu_circo(x))
df2_2020['code_chef_lieu'] = df2_2020.reg.apply(lambda x: comp_code_cl(x))
df2_2020['chef_lieu_libelle'] = df2_2020.reg.apply(lambda x: comp_cl_libelle(x))
df2_2020.head()

,typecom,com,reg,dep,libelle,EU_circo,code_chef_lieu,chef_lieu_libelle
0,COM,01001,84,01,L'Abergement-Clémenciat,Sud-Est,69123,Lyon
1,COM,01002,84,01,L'Abergement-de-Varey,Sud-Est,69123,Lyon
2,COM,01004,84,01,Ambérieu-en-Bugey,Sud-Est,69123,Lyon
3,COM,01005,84,01,Ambérieux-en-Dombes,Sud-Est,69123,Lyon
4,COM,01006,84,01,Ambléon,Sud-Est,69123,Lyon


In [9]:
df2_2020.shape

(35013, 8)

In [10]:
# Fusion des données geo et GPS
df2 = pd.merge(df2_2020,geo,how='left',left_on='com',right_on='COM')

In [11]:
df2.head()

,typecom,com,reg,dep,libelle,EU_circo,code_chef_lieu,chef_lieu_libelle,COM,LIBELLE_DEP,LIBELLE_REG,Code_postal,latitude,longitude
0,COM,01001,84,01,L'Abergement-Clémenciat,Sud-Est,69123,Lyon,01001,Ain,Auvergne-Rhône-Alpes,01400,46.153721,4.925850
1,COM,01002,84,01,L'Abergement-de-Varey,Sud-Est,69123,Lyon,01002,Ain,Auvergne-Rhône-Alpes,01640,46.009606,5.428088
2,COM,01004,84,01,Ambérieu-en-Bugey,Sud-Est,69123,Lyon,01004,Ain,Auvergne-Rhône-Alpes,01500,45.961049,5.372275
3,COM,01005,84,01,Ambérieux-en-Dombes,Sud-Est,69123,Lyon,01005,Ain,Auvergne-Rhône-Alpes,01330,45.996164,4.911967
4,COM,01006,84,01,Ambléon,Sud-Est,69123,Lyon,01006,Ain,Auvergne-Rhône-Alpes,01300,45.749886,5.594585


In [12]:
# Suppression des doublons
df2 = df2.drop_duplicates(subset='com',keep='first')
# Vérification de la suppression
df2.loc[df2.duplicated(subset='com')]

,typecom,com,reg,dep,libelle,EU_circo,code_chef_lieu,chef_lieu_libelle,COM,LIBELLE_DEP,LIBELLE_REG,Code_postal,latitude,longitude


In [13]:
# Suppression des colonnes non utiles pour le prochain merge
df2_base = df2.drop(labels=['typecom','COM'],axis='columns')
df2_base.shape

(35013, 12)

In [14]:
# Verification des nan
df2_base.loc[df2_base.com.isna()]

,com,reg,dep,libelle,EU_circo,code_chef_lieu,chef_lieu_libelle,LIBELLE_DEP,LIBELLE_REG,Code_postal,latitude,longitude


## Informations démographiques par ville, âge, sexe et mode de vie

<u>donnees_communes.csv : </u> Données INSEE du recensement de 2020

* CODDEP
* CODCOM
* PTOT

<u>mayotte_pop_legale.csv : </u> Données INSEE du recensement de 2017

* CODDEP
* CODCOM
* PTOT


In [15]:
# Lecture du fichier de données population
population = pd.read_csv('donnees_communes.csv',sep=';',usecols=['CODDEP','CODCOM','PTOT'])
population.head()

,CODDEP,CODCOM,PTOT
0,01,1,821
1,01,2,268
2,01,4,14662
3,01,5,1806
4,01,6,113


In [16]:
population_may = pd.read_csv('mayotte_pop_legale.csv',sep=';',index_col=False,dtype={'CODDEP':'str'},usecols=['CODDEP','CODCOM','PTOT'])
population_may.head()

,CODDEP,CODCOM,PTOT
0,976,601,5384
1,976,602,14211
2,976,603,10529
3,976,604,6503
4,976,605,8616


In [17]:
population_may.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODDEP  17 non-null     object
 1   CODCOM  17 non-null     int64 
 2   PTOT    17 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 536.0+ bytes


In [18]:
population = population.append(population_may)

In [19]:
population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34997 entries, 0 to 16
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODDEP  34997 non-null  object
 1   CODCOM  34997 non-null  int64 
 2   PTOT    34997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [20]:
# Construction du code commune pour faciliter le merge
def Cons_code_com(code_dep,code_com):
    if len(code_dep) == 1:
        code_dep = '0' + code_dep
    if code_dep[0:2] == '97':
        code_dep = '97'
    if code_com < 10:
        codecom = code_dep + '00' + str(code_com)
    elif code_com < 100:
        codecom = code_dep + '0' + str(code_com)
    else:
        codecom = code_dep + str(code_com)
    return codecom   

population['COM'] = population.apply(lambda x : Cons_code_com(x.CODDEP,x.CODCOM),axis=1)


In [21]:
# Affichage des premières lignes
population

,CODDEP,CODCOM,PTOT,COM
0,01,1,821,01001
1,01,2,268,01002
2,01,4,14662,01004
3,01,5,1806,01005
4,01,6,113,01006
...,...,...,...,...
12,976,613,6586,97613
13,976,614,10393,97614
14,976,615,11802,97615
15,976,616,11619,97616


In [22]:
# Ajout des données pour Paris, Lyon et Marseille : aggregation des données par arrondissement
# pour cohérence avec dataset entreprises et salaires
population.loc[population.CODDEP=='75'].PTOT.sum()
pop_paris = population.loc[population.CODDEP=='75'].PTOT.sum()

pop_temp = population.loc[population.CODDEP=='69']
liste_l = ['69381','69382','69383','69384','69385','69386','69387','69388','69389']
pop_lyon = 0
for i in liste_l:
    pop_lyon = pop_lyon + int(pop_temp.loc[pop_temp.COM==i].PTOT.values)

pop_temp = population.loc[population.CODDEP=='13']
liste_m = ['13201','13202','13203','13204','13205','13206','13207',
           '13208','13209','13210','13211','13212','13213','13214','13215','13216']
pop_marseille = 0
for i in liste_m:
    pop_marseille = pop_marseille + int(pop_temp.loc[pop_temp.COM==i].PTOT.values)

dict = {'CODDEP':['75','69','13'],'CODCOM':[0,0,0],'PTOT':[pop_paris,pop_lyon,pop_marseille],'COM':['75056','69123','13055']}
df_temp = pd.DataFrame(data=dict)
df_temp
population = population.append(df_temp)

In [23]:
# tri des codes communes et reindexation du fichier
population = population.sort_values('COM')
population = population.reset_index(drop=True)

In [24]:
# Affichage des infos de la DF
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODDEP  35000 non-null  object
 1   CODCOM  35000 non-null  int64 
 2   PTOT    35000 non-null  int64 
 3   COM     35000 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.1+ MB


In [25]:
# Affichage de la dimension
population.shape

(35000, 4)

In [26]:
# Verification des nan
population.isna().sum()

CODDEP    0
CODCOM    0
PTOT      0
COM       0
dtype: int64

In [27]:
# Fusion des données geo et population
all_content = pd.merge(df2_base,population,how='inner',right_on='COM',left_on='com')

In [28]:
# Suppression des nan pour Lyon et Marseille
all_content.loc[all_content.loc[all_content.COM=='69123'].index,'longitude'] = np.float64(all_content.loc[all_content.loc[all_content.COM=='69381'].index,'longitude'].values)
all_content.loc[all_content.loc[all_content.COM=='69123'].index,'latitude'] = np.float64(all_content.loc[all_content.loc[all_content.COM=='69381'].index,'latitude'].values)
all_content.loc[all_content.loc[all_content.COM=='69123'].index,'Code_postal'] = '69000'

all_content.loc[all_content.loc[all_content.COM=='13055'].index,'longitude'] = np.float64(all_content.loc[all_content.loc[all_content.COM=='13201'].index,'longitude'].values)
all_content.loc[all_content.loc[all_content.COM=='13055'].index,'latitude'] = np.float64(all_content.loc[all_content.loc[all_content.COM=='13201'].index,'latitude'].values)
all_content.loc[all_content.loc[all_content.COM=='13055'].index,'Code_postal'] = '13000'


In [29]:
# Affichage des infos de la DF
all_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35000 entries, 0 to 34999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   com                35000 non-null  object 
 1   reg                35000 non-null  object 
 2   dep                35000 non-null  object 
 3   libelle            35000 non-null  object 
 4   EU_circo           35000 non-null  object 
 5   code_chef_lieu     35000 non-null  object 
 6   chef_lieu_libelle  35000 non-null  object 
 7   LIBELLE_DEP        35000 non-null  object 
 8   LIBELLE_REG        35000 non-null  object 
 9   Code_postal        35000 non-null  object 
 10  latitude           34992 non-null  float64
 11  longitude          34992 non-null  float64
 12  CODDEP             35000 non-null  object 
 13  CODCOM             35000 non-null  int64  
 14  PTOT               35000 non-null  int64  
 15  COM                35000 non-null  object 
dtypes: float64(2), int64(2

In [30]:
# Suppression des colonnes en doublon
all_content = all_content.drop(columns=['CODDEP','CODCOM','com'])

In [31]:
# Affichage de la DF
all_content.head()

,reg,dep,libelle,EU_circo,code_chef_lieu,chef_lieu_libelle,LIBELLE_DEP,LIBELLE_REG,Code_postal,latitude,longitude,PTOT,COM
0,84,01,L'Abergement-Clémenciat,Sud-Est,69123,Lyon,Ain,Auvergne-Rhône-Alpes,01400,46.153721,4.925850,821,01001
1,84,01,L'Abergement-de-Varey,Sud-Est,69123,Lyon,Ain,Auvergne-Rhône-Alpes,01640,46.009606,5.428088,268,01002
2,84,01,Ambérieu-en-Bugey,Sud-Est,69123,Lyon,Ain,Auvergne-Rhône-Alpes,01500,45.961049,5.372275,14662,01004
3,84,01,Ambérieux-en-Dombes,Sud-Est,69123,Lyon,Ain,Auvergne-Rhône-Alpes,01330,45.996164,4.911967,1806,01005
4,84,01,Ambléon,Sud-Est,69123,Lyon,Ain,Auvergne-Rhône-Alpes,01300,45.749886,5.594585,113,01006


In [32]:
# On renomme les colonnes 
dict_columns = {'COM': 'CODGEO','reg':'num_region','dep':'num_dep','libelle':'nom_commune',
                'EU_circo':'EU_circo','code_chef_lieu': 'num_chef_lieu','chef_lieu_libelle':'nom_chef_lieu',
                'LIBELLE_DEP': 'nom_dep','LIBELLE_REG':'nom_region','Code_postal':'code_postal','latitude':'latitude',
                'longitude':'longitude','PTOT':'pop_tot'}

all_content = all_content.rename(columns=dict_columns)

In [33]:
# On recupère le nom des colonnes dans une liste
all_content_columns = all_content.columns.tolist()
# Re-organisation des colonnes
all_content_geo_columns = ['CODGEO'] + ['EU_circo'] + all_content_columns[0:3] + all_content_columns[6:8] + all_content_columns[4:6] + all_content_columns[-5:-1]
all_content_geo = all_content[all_content_geo_columns]
# Affichage des premières lignes
all_content_geo.head()

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,latitude,longitude,pop_tot
0,01001,Sud-Est,84,01,L'Abergement-Clémenciat,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01400,46.153721,4.925850,821
1,01002,Sud-Est,84,01,L'Abergement-de-Varey,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01640,46.009606,5.428088,268
2,01004,Sud-Est,84,01,Ambérieu-en-Bugey,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01500,45.961049,5.372275,14662
3,01005,Sud-Est,84,01,Ambérieux-en-Dombes,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01330,45.996164,4.911967,1806
4,01006,Sud-Est,84,01,Ambléon,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01300,45.749886,5.594585,113


In [34]:
# Taille de la DataFrame
all_content_geo.shape

(35000, 13)

In [35]:
# Recherche des duplications
all_content_dupli = all_content_geo.loc[all_content.duplicated(subset='CODGEO')]
# Affichage des duplications
all_content_dupli

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,latitude,longitude,pop_tot


In [36]:
# Recherche des nan
all_content_geo.isna().sum()

CODGEO           0
EU_circo         0
num_region       0
num_dep          0
nom_commune      0
nom_dep          0
nom_region       0
num_chef_lieu    0
nom_chef_lieu    0
code_postal      0
latitude         8
longitude        8
pop_tot          0
dtype: int64

In [37]:
# Affichage des lignes avec nan
all_content_geo.loc[all_content_geo.latitude.isna()]

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,latitude,longitude,pop_tot
35,01039,Sud-Est,84,01,Béon,Ain,Auvergne-Rhône-Alpes,69123,Lyon,nan,NaN,NaN,446
467,02077,Nord-Ouest,32,02,Berzy-le-Sec,Aisne,Hauts-de-France,59350,Lille,nan,NaN,NaN,389
3062,09255,Sud-Ouest,76,09,Saint-Amans,Ariège,Occitanie,31555,Toulouse,nan,NaN,NaN,41
5325,16140,Sud-Ouest,75,16,Fontclaireau,Charente,Nouvelle-Aquitaine,33063,Bordeaux,nan,NaN,NaN,441
17720,50015,Nord-Ouest,28,50,Annoville,Manche,Normandie,76540,Rouen,nan,NaN,NaN,654
28285,71492,Est,27,71,Saint-Ythaire,Saône-et-Loire,Bourgogne-Franche-Comté,21231,Dijon,nan,NaN,NaN,129
32653,85037,Ouest,52,85,Breuil-Barret,Vendée,Pays de la Loire,44109,Nantes,nan,NaN,NaN,595
32665,85053,Ouest,52,85,La Chapelle-aux-Lys,Vendée,Pays de la Loire,44109,Nantes,nan,NaN,NaN,262


## Meta dossier complet INSEE 2020

<u>meta_dossier_complet.csv</u>

* COD_VAR : code de la variable
* LIB_VAR : libellé de la variable
* LIB_VAR_LONG : libellé long de la variable
* COD_MOD : Modalité des prises par les variables
* LIB_MOD : Libellé des modalités
* TYPE_VAR : Type de variables
* LONG_VAR : longueur des variables
* THEME : Thème associé à la variable
* SOURCE : Source utilisée pour la variable

In [38]:
# Lecture du fichier csv contenant les nom des variables par categorie
df5 = pd.read_csv('meta_dossier_complet.csv',sep=';',low_memory=False)
df5.COD_VAR.unique()

array(['P19_POP', 'P19_POP0014', 'P19_POP1529', ..., 'AJCSUH23',
       'AJCSLIT23', 'CODGEO'], dtype=object)

In [39]:
# Suppression des variables non utiles pour notre étude
df5_copie = df5.copy()
list_index = []
for i in df5.COD_VAR.values:
    if (i[1:3] in ['08','13','99','90','82','75','68']) | (i[-2:] in ['18','17','16','15','14','13','12']) | (i[0:4]=='NAIS') | (i[0:4]=='DECE'):
        list_index.append(int(df5_copie.loc[df5.COD_VAR==i].index.values))
# list_index
list_index = list_index + df5_copie.loc[df5.COD_VAR=='CODGEO'].index.values.tolist() + df5.loc[(df5['THEME']=='Tourisme') | (df5['THEME']=='Naissances et décès domiciliés')].index.values.tolist()
# Suppression des lignes
df5 = df5_copie.drop(labels=list_index,axis='index')

In [40]:
df5.THEME.value_counts()

Caractéristiques de l'emploi au sens du recensement            127
Évolution et structure de la population                        102
Caractéristiques des établissements                             94
Logement                                                        83
Couples - Familles - Ménages                                    80
Démographie des entreprises                                     74
Population active, emploi et chômage au sens du recensement     58
Diplômes - Formation                                            54
Revenus et pauvreté des ménages                                 27
Salaires et revenus d'activité                                  24
Name: THEME, dtype: int64

In [41]:
# regroupent par categories
theme_var=df5.groupby(['THEME','COD_VAR','LIB_VAR']).size()

# Augmentation du nombre de lignes à afficher 
nbre_ligne = theme_var.shape[0]
pd.set_option("display.max_rows",nbre_ligne)

# Affichage des variables par categorie
theme_var=pd.DataFrame(theme_var)
theme_var

0
THEME                                              COD_VAR                       LIB_VAR                                              
Caractéristiques de l'emploi au sens du recense... C19_ACT1564                   Actifs 15-64 ans en 2019 (compl)                    1
                                                   C19_ACT1564_CS1               Actifs 15-64 ans Agriculteurs exploitants en 20...  1
                                                   C19_ACT1564_CS2               Actifs 15-64 ans Artisans, Comm., Chefs entr. e...  1
                                                   C19_ACT1564_CS3               Actifs 15-64 ans Cadres, Prof. intel. sup. en 2...  1
                                                   C19_ACT1564_CS4               Actifs 15-64 ans Prof. intermédiaires en 2019 (...  1
                                                   C19_ACT1564_CS5               Actifs 15-64 ans Employés en 2019 (compl)           1
                                                   C19_ACT1564_CS6               Actifs 15-64 ans Ouvriers en 2019 (compl)           1
                                                   C19_ACTOCC1564                Actifs occupés 15-64 ans en 2019 (compl)            1
                                                   C19_ACTOCC1564_CS1            Actifs occ 15-64 ans Agriculteurs exploitants e...  1
                                                   C19_ACTOCC1564_CS2            Actifs occ 15-64 ans Artisans, Comm., Chefs ent...  1
                                                   C19_ACTOCC1564_CS3            Actifs occ 15-64 ans Cadres Prof. intel. sup. e...  1
                                                   C19_ACTOCC1564_CS4            Actifs occ 15-64 ans Prof. intermédiaires en 20...  1
                                                   C19_ACTOCC1564_CS5            Actifs occupés 15-64 ans Employés en 2019 (compl)   1
                                                   C19_ACTOCC1564_CS6            Actifs occupés 15-64 ans Ouvriers en 2019 (compl)   1
                                                   C19_AGRILT_F                  Emplois femmes au LT Agriculture en 2019 (compl)    1
                                                   C19_AGRILT_FNSAL              Emplois non-sal femmes au LT Agriculture en 201...  1
                                                   C19_AGRILT_FSAL               Emplois salariés femmes au LT Agriculture en 20...  1
                                                   C19_AGRILT_NSAL               Emplois non-salariés au LT Agriculture en 2019 ...  1
                                                   C19_AGRILT_SAL                Emplois salariés au LT Agriculture en 2019 (compl)  1
                                                   C19_APESASLT_F                Emplois femmes au LT Adm publique, Enseignement...  1
                                                   C19_APESASLT_FNSAL            Emplois non-sal femmes au LT Adm publique, Ense...  1
                                                   C19_APESASLT_FSAL             Emplois salariés femmes au LT Adm publique, Ens...  1
                                                   C19_APESASLT_NSAL             Emplois non-salariés au LT Adm publique, Enseig...  1
                                                   C19_APESASLT_SAL              Emplois salariés au LT Adm publique, Enseigneme...  1
                                                   C19_CONSTLT_F                 Emplois femmes au LT Construction en 2019 (compl)   1
                                                   C19_CONSTLT_FNSAL             Emplois non-sal femmes au LT Construction en 20...  1
                                                   C19_CONSTLT_FSAL              Emplois salariés femmes au LT Construction en 2...  1
                                                   C19_CONSTLT_NSAL              Emplois non-salariés au LT Construction en 2019...  1
                                                   C19_CONSTLT_SAL               Em

In [42]:
# Creation des listes de categories par theme si besoin
familles_pop_columns = df5.loc[df5['THEME']=='Couples - Familles - Ménages']['COD_VAR'].values
dip_formation = df5.loc[df5['THEME']=="Diplômes - Formation"]['COD_VAR'].values
pop_act_emploi = df5.loc[df5['THEME']=="Population active, emploi et chômage au sens du recensement"]['COD_VAR'].values
salaires_columns = df5.loc[df5['THEME']=="Salaires et revenus d'activité"]['COD_VAR'].values
caract_ets_columns = df5.loc[df5['THEME']=='Caractéristiques des établissements']['COD_VAR'].values
carac_emp_recens = df5.loc[df5['THEME']=="Caractéristiques de l'emploi au sens du recensement"]['COD_VAR'].values
evol_struct_pop = df5.loc[df5['THEME']=="Évolution et structure de la population"]['COD_VAR'].values
Logement = df5.loc[df5['THEME']=="Logement"]['COD_VAR'].values
demo_entr = df5.loc[df5['THEME']=="Démographie des entreprises"]['COD_VAR'].values
rev_pauv_menages = df5.loc[df5['THEME']=="Revenus et pauvreté des ménages"]['COD_VAR'].values

# Liste de toutes les categories jugées utiles pour le projet
liste_var = ['CODGEO'] + df5.COD_VAR.values.tolist()

# <font color= 'blue' size=6>Creation du fichier csv All_content_2020 </font>


## <u>dossier complet INSEE 2020</u>

<font color= 'brown' size=3>"dossier_complet.csv"</font>


## <u>Thèmes</u>

* Caractéristiques de l'emploi au sens du recensement             (Nombre variables : 127)
* Évolution et structure de la population                         (Nombre variables : 102
* Caractéristiques des établissements                             (Nombre variables : 94)
* Logement                                                        (Nombre variables : 83)
* Couples - Familles - Ménages                                    (Nombre variables : 80)
* Démographie des entreprises                                     (Nombre variables : 74)
* Population active, emploi et chômage au sens du recensement     (Nombre variables : 58)
* Diplômes - Formation                                            (Nombre variables : 54)
* Revenus et pauvreté des ménages                                 (Nombre variables : 27)
* Salaires et revenus d'activité                                  (Nombre variables : 24)

<mark><font color='brown'>*Voir "meta_dossier_complet.xlsx" pour plus de détails*</font></mark>



In [43]:
# Lecture du fichier csv
df6 = pd.read_csv('dossier_complet.csv',sep=';',usecols=liste_var,low_memory=False)
# affichage de la dimension
df6.shape

(35000, 724)

In [44]:
# Affichage des premières lignes
df6.head()

,CODGEO,P19_POP,P19_POP1529,P19_POP3044,P19_POP4559,P19_POP6074,P19_POP7589,P19_POP90P,P19_POPH,P19_H1529,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
0,01001,779.0,102.468339,131.523972,194.112147,131.050309,56.014471,5.322133,397.885214,55.349857,...,9.0,0.0,1.0,2.0,0.0,1.0,0.0,3.0,1.0,1.0
1,01002,256.0,29.609476,74.028483,38.684019,35.381928,17.582020,2.067723,129.517516,10.625566,...,5.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0
2,01004,14134.0,3076.783602,2730.398468,2391.347109,2054.139468,1044.007592,195.156716,6790.643609,1565.330686,...,178.0,11.0,27.0,41.0,9.0,5.0,10.0,27.0,27.0,21.0
3,01005,1751.0,268.250842,362.580808,380.267677,272.181257,111.034231,9.826038,877.465208,133.634119,...,19.0,1.0,7.0,6.0,0.0,0.0,0.0,4.0,1.0,0.0
4,01006,112.0,16.290909,15.272727,29.527273,27.490909,10.181818,1.018182,63.127273,12.218182,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [45]:
# Affichage des informations
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Columns: 724 entries, CODGEO to ETCRU19
dtypes: float64(634), int64(62), object(28)
memory usage: 193.3+ MB


In [46]:
# merge des variables par categories avec donées geo et population
all_content_pop_df6 = pd.merge(df6,all_content_geo,how='left',on='CODGEO')

In [47]:
# Suppression des arrondissement de Paris
index_temp = all_content_pop_df6.loc[all_content_pop_df6.nom_commune.str[-14:]=='Arrondissement'].index
all_content_pop_df6 = all_content_pop_df6.drop(labels=index_temp,axis='index')

In [48]:
# On recupère le nom des colonnes dans une liste
all_content_pop_df6_columns = all_content_pop_df6.columns.tolist()
# Re-organisation des colonnes
all_content_2020_columns =  all_content_geo_columns + ['SUPERF'] + all_content_pop_df6_columns[1:-12] 
all_content_2020 = all_content_pop_df6[all_content_2020_columns]
# Affichage des premières lignes
all_content_2020.head()

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
0,01001,Sud-Est,84,01,L'Abergement-Clémenciat,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01400,...,9.0,0.0,1.0,2.0,0.0,1.0,0.0,3.0,1.0,1.0
1,01002,Sud-Est,84,01,L'Abergement-de-Varey,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01640,...,5.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0
2,01004,Sud-Est,84,01,Ambérieu-en-Bugey,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01500,...,178.0,11.0,27.0,41.0,9.0,5.0,10.0,27.0,27.0,21.0
3,01005,Sud-Est,84,01,Ambérieux-en-Dombes,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01330,...,19.0,1.0,7.0,6.0,0.0,0.0,0.0,4.0,1.0,0.0
4,01006,Sud-Est,84,01,Ambléon,Ain,Auvergne-Rhône-Alpes,69123,Lyon,01300,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [49]:
all_content_2020.iloc[:,0:14].columns

Index(['CODGEO', 'EU_circo', 'num_region', 'num_dep', 'nom_commune', 'nom_dep',
       'nom_region', 'num_chef_lieu', 'nom_chef_lieu', 'code_postal',
       'latitude', 'longitude', 'pop_tot', 'SUPERF'],
      dtype='object')

In [50]:
all_content_2020.iloc[:,-12:].columns

Index(['ETCOQ20', 'ETCRU20', 'ETCTOT19', 'ETCBE19', 'ETCFZ19', 'ETCGI19',
       'ETCJZ19', 'ETCKZ19', 'ETCLZ19', 'ETCMN19', 'ETCOQ19', 'ETCRU19'],
      dtype='object')

In [51]:
# Affichage dimension de la DF
all_content_2020.shape

(34955, 737)

In [52]:
# Affichage des nan pour analyse
temp = all_content_2020.isna().sum()
temp

CODGEO         0
EU_circo       0
num_region     0
num_dep        0
nom_commune    0
              ..
ETCKZ19        1
ETCLZ19        1
ETCMN19        1
ETCOQ19        1
ETCRU19        1
Length: 737, dtype: int64

In [53]:
temp[(temp>31500) & (temp<34955)]

P19_RP_ELEC      34843
P19_RP_EAUCH     34843
P19_RP_BDWC      34843
P19_RP_CHOS      34843
P19_RP_CLIM      34843
P19_RP_TTEGOU    34843
P19_RP_HABFOR    34843
P19_RP_CASE      34843
P19_RP_MIBOIS    34843
P19_RP_MIDUR     34843
dtype: int64

In [54]:
# Renseigné pour Outre-mer uniquement
all_content_2020.loc[all_content_2020.P19_RP_ELEC.notna()]

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
34871,97101,Outre-Mer,01,971,Les Abymes,Guadeloupe,Guadeloupe,97105,Basse-Terre,97142,...,749.0,64.0,81.0,240.0,21.0,15.0,20.0,163.0,81.0,64.0
34872,97102,Outre-Mer,01,971,Anse-Bertrand,Guadeloupe,Guadeloupe,97105,Basse-Terre,97121,...,63.0,7.0,10.0,16.0,0.0,0.0,3.0,10.0,11.0,6.0
34873,97103,Outre-Mer,01,971,Baie-Mahault,Guadeloupe,Guadeloupe,97105,Basse-Terre,97122,...,863.0,97.0,69.0,232.0,24.0,26.0,45.0,204.0,97.0,69.0
34874,97104,Outre-Mer,01,971,Baillif,Guadeloupe,Guadeloupe,97105,Basse-Terre,97123,...,59.0,6.0,11.0,13.0,0.0,1.0,0.0,7.0,14.0,7.0
34875,97105,Outre-Mer,01,971,Basse-Terre,Guadeloupe,Guadeloupe,97105,Basse-Terre,97100,...,143.0,15.0,9.0,59.0,0.0,0.0,2.0,21.0,30.0,7.0
34876,97106,Outre-Mer,01,971,Bouillante,Guadeloupe,Guadeloupe,97105,Basse-Terre,97125,...,102.0,9.0,16.0,24.0,5.0,1.0,6.0,15.0,19.0,7.0
34877,97107,Outre-Mer,01,971,Capesterre-Belle-Eau,Guadeloupe,Guadeloupe,97105,Basse-Terre,97130,...,209.0,19.0,24.0,62.0,9.0,2.0,2.0,38.0,38.0,15.0
34878,97108,Outre-Mer,01,971,Capesterre-de-Marie-Galante,Guadeloupe,Guadeloupe,97105,Basse-Terre,97140,...,17.0,0.0,1.0,8.0,0.0,0.0,1.0,5.0,1.0,1.0
34879,97109,Outre-Mer,01,971,Gourbeyre,Guadeloupe,Guadeloupe,97105,Basse-Terre,97113,...,74.0,2.0,8.0,15.0,1.0,0.0,1.0,15.0,21.0,11.0
34880,97110,Outre-Mer,01,971,La Désirade,Guadeloupe,Guadeloupe,97105,Basse-Terre,97127,...,13.0,1.0,0.0,3.0,0.0,0.0,0.0,2.0,3.0,4.0


In [55]:
# Données salariales pour environ 5k communes
temp[(temp>129) & (temp<34843)]

SNHM20       29579
SNHMC20      29579
SNHMP20      29579
SNHME20      29579
SNHMO20      29579
SNHMF20      29579
SNHMFC20     29579
SNHMFP20     29579
SNHMFE20     29579
SNHMFO20     29579
SNHMH20      29579
SNHMHC20     29579
SNHMHP20     29579
SNHMHE20     29579
SNHMHO20     29579
SNHM1820     29579
SNHM2620     29579
SNHM5020     29579
SNHMF1820    29579
SNHMF2620    29579
SNHMF5020    29579
SNHMH1820    29579
SNHMH2620    29579
SNHMH5020    29579
dtype: int64

In [56]:
temp[(temp>99) & (temp<130)]

P19_RP_SDB         129
P19_RP_CCCOLL      129
P19_RP_CCIND       129
P19_RP_CINDELEC    129
dtype: int64

In [57]:
# Non renseigné pour Outre-mer
all_content_2020.loc[all_content_2020.P19_RP_SDB.isna()]

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
34871,97101,Outre-Mer,01,971,Les Abymes,Guadeloupe,Guadeloupe,97105,Basse-Terre,97142,...,749.0,64.0,81.0,240.0,21.0,15.0,20.0,163.0,81.0,64.0
34872,97102,Outre-Mer,01,971,Anse-Bertrand,Guadeloupe,Guadeloupe,97105,Basse-Terre,97121,...,63.0,7.0,10.0,16.0,0.0,0.0,3.0,10.0,11.0,6.0
34873,97103,Outre-Mer,01,971,Baie-Mahault,Guadeloupe,Guadeloupe,97105,Basse-Terre,97122,...,863.0,97.0,69.0,232.0,24.0,26.0,45.0,204.0,97.0,69.0
34874,97104,Outre-Mer,01,971,Baillif,Guadeloupe,Guadeloupe,97105,Basse-Terre,97123,...,59.0,6.0,11.0,13.0,0.0,1.0,0.0,7.0,14.0,7.0
34875,97105,Outre-Mer,01,971,Basse-Terre,Guadeloupe,Guadeloupe,97105,Basse-Terre,97100,...,143.0,15.0,9.0,59.0,0.0,0.0,2.0,21.0,30.0,7.0
34876,97106,Outre-Mer,01,971,Bouillante,Guadeloupe,Guadeloupe,97105,Basse-Terre,97125,...,102.0,9.0,16.0,24.0,5.0,1.0,6.0,15.0,19.0,7.0
34877,97107,Outre-Mer,01,971,Capesterre-Belle-Eau,Guadeloupe,Guadeloupe,97105,Basse-Terre,97130,...,209.0,19.0,24.0,62.0,9.0,2.0,2.0,38.0,38.0,15.0
34878,97108,Outre-Mer,01,971,Capesterre-de-Marie-Galante,Guadeloupe,Guadeloupe,97105,Basse-Terre,97140,...,17.0,0.0,1.0,8.0,0.0,0.0,1.0,5.0,1.0,1.0
34879,97109,Outre-Mer,01,971,Gourbeyre,Guadeloupe,Guadeloupe,97105,Basse-Terre,97113,...,74.0,2.0,8.0,15.0,1.0,0.0,1.0,15.0,21.0,11.0
34880,97110,Outre-Mer,01,971,La Désirade,Guadeloupe,Guadeloupe,97105,Basse-Terre,97127,...,13.0,1.0,0.0,3.0,0.0,0.0,0.0,2.0,3.0,4.0


In [58]:
temp[(temp>69) & (temp<100)]

NBMENFISC20        71
NBPERSMENFISC20    71
MED20              71
PIMP20             71
TP6020             71
TP60AGE120         71
TP60AGE220         71
TP60AGE320         71
TP60AGE420         71
TP60AGE520         71
TP60AGE620         71
TP60TOL120         71
TP60TOL220         71
PACT20             71
PTSA20             71
PCHO20             71
PBEN20             71
PPEN20             71
PPAT20             71
PPSOC20            71
PPFAM20            71
PPMINI20           71
PPLOGT20           71
PIMPOT20           71
D120               71
D920               71
RD20               71
dtype: int64

In [59]:
# Non renseigné pour Outre-mer
all_content_2020.loc[all_content_2020.MED20.isna()]

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
34871,97101,Outre-Mer,01,971,Les Abymes,Guadeloupe,Guadeloupe,97105,Basse-Terre,97142,...,749.0,64.0,81.0,240.0,21.0,15.0,20.0,163.0,81.0,64.0
34872,97102,Outre-Mer,01,971,Anse-Bertrand,Guadeloupe,Guadeloupe,97105,Basse-Terre,97121,...,63.0,7.0,10.0,16.0,0.0,0.0,3.0,10.0,11.0,6.0
34873,97103,Outre-Mer,01,971,Baie-Mahault,Guadeloupe,Guadeloupe,97105,Basse-Terre,97122,...,863.0,97.0,69.0,232.0,24.0,26.0,45.0,204.0,97.0,69.0
34874,97104,Outre-Mer,01,971,Baillif,Guadeloupe,Guadeloupe,97105,Basse-Terre,97123,...,59.0,6.0,11.0,13.0,0.0,1.0,0.0,7.0,14.0,7.0
34875,97105,Outre-Mer,01,971,Basse-Terre,Guadeloupe,Guadeloupe,97105,Basse-Terre,97100,...,143.0,15.0,9.0,59.0,0.0,0.0,2.0,21.0,30.0,7.0
34876,97106,Outre-Mer,01,971,Bouillante,Guadeloupe,Guadeloupe,97105,Basse-Terre,97125,...,102.0,9.0,16.0,24.0,5.0,1.0,6.0,15.0,19.0,7.0
34877,97107,Outre-Mer,01,971,Capesterre-Belle-Eau,Guadeloupe,Guadeloupe,97105,Basse-Terre,97130,...,209.0,19.0,24.0,62.0,9.0,2.0,2.0,38.0,38.0,15.0
34878,97108,Outre-Mer,01,971,Capesterre-de-Marie-Galante,Guadeloupe,Guadeloupe,97105,Basse-Terre,97140,...,17.0,0.0,1.0,8.0,0.0,0.0,1.0,5.0,1.0,1.0
34879,97109,Outre-Mer,01,971,Gourbeyre,Guadeloupe,Guadeloupe,97105,Basse-Terre,97113,...,74.0,2.0,8.0,15.0,1.0,0.0,1.0,15.0,21.0,11.0
34880,97110,Outre-Mer,01,971,La Désirade,Guadeloupe,Guadeloupe,97105,Basse-Terre,97127,...,13.0,1.0,0.0,3.0,0.0,0.0,0.0,2.0,3.0,4.0


In [60]:
temp[(temp>17) & (temp<70)]

P19_POP1519_COUPLE    18
C19_MEN_CS1           18
C19_PMEN_CS1          18
C19_HMONO             18
ETBE20                18
ETBE020               18
ETOQ020               18
ETAZ120               18
ETBE120               18
ETAZ1020              18
ETBE1020              18
ETAZ2020              18
ETBE2020              18
ETGU2020              18
ETGZ2020              18
ETTEF5020             18
ETAZ5020              18
ETBE5020              18
ETFZ5020              18
ETGU5020              18
ETGZ5020              18
ETOQ5020              18
ETPAZ20               18
ETPBE20               18
ETPAZ120              18
ETPBE120              18
ETPAZ1020             18
ETPBE1020             18
ETPAZ2020             18
ETPBE2020             18
ETPGU2020             18
ETPGZ2020             18
ETPTEF5020            18
ETPAZ5020             18
ETPBE5020             18
ETPFZ5020             18
ETPGU5020             18
ETPGZ5020             18
ETPOQ5020             18
ETPTEFCP20            18


In [61]:
# Données manquantes pour Mayotte principalement
all_content_2020.loc[all_content_2020.P19_POP1519_COUPLE.isna()]

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
4884,14666,Nord-Ouest,28,14,Sannerville,Calvados,Normandie,76540,Rouen,14940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34983,97601,Outre-Mer,06,976,Acoua,Mayotte,Mayotte,97608,Dzaoudzi,97630,...,23.0,1.0,5.0,11.0,0.0,0.0,0.0,4.0,1.0,1.0
34984,97602,Outre-Mer,06,976,Bandraboua,Mayotte,Mayotte,97608,Dzaoudzi,97650,...,61.0,1.0,6.0,41.0,1.0,0.0,0.0,3.0,8.0,1.0
34985,97603,Outre-Mer,06,976,Bandrele,Mayotte,Mayotte,97608,Dzaoudzi,97660,...,52.0,2.0,7.0,32.0,1.0,0.0,0.0,3.0,5.0,2.0
34986,97604,Outre-Mer,06,976,Bouéni,Mayotte,Mayotte,97608,Dzaoudzi,97620,...,42.0,1.0,6.0,21.0,0.0,1.0,0.0,4.0,6.0,3.0
34987,97605,Outre-Mer,06,976,Chiconi,Mayotte,Mayotte,97608,Dzaoudzi,97670,...,31.0,2.0,2.0,22.0,1.0,0.0,0.0,2.0,2.0,0.0
34988,97606,Outre-Mer,06,976,Chirongui,Mayotte,Mayotte,97608,Dzaoudzi,97620,...,61.0,3.0,4.0,40.0,0.0,0.0,1.0,4.0,8.0,1.0
34989,97607,Outre-Mer,06,976,Dembeni,Mayotte,Mayotte,97608,Dzaoudzi,97660,...,52.0,3.0,9.0,27.0,0.0,0.0,0.0,3.0,4.0,6.0
34990,97608,Outre-Mer,06,976,Dzaoudzi,Mayotte,Mayotte,97608,Dzaoudzi,97615,...,104.0,2.0,11.0,65.0,3.0,2.0,0.0,6.0,8.0,7.0
34991,97609,Outre-Mer,06,976,Kani-Kéli,Mayotte,Mayotte,97608,Dzaoudzi,97625,...,14.0,1.0,2.0,7.0,0.0,0.0,0.0,1.0,1.0,2.0


In [62]:
temp[(temp>1) & (temp<18)]

latitude                          8
longitude                         8
SUPERF                           17
P19_POP                          17
P19_POP1529                      17
P19_POP3044                      17
P19_POP4559                      17
P19_POP6074                      17
P19_POP7589                      17
P19_POP90P                       17
P19_POPH                         17
P19_H1529                        17
P19_H3044                        17
P19_H4559                        17
P19_H6074                        17
P19_H7589                        17
P19_H90P                         17
P19_H0019                        17
P19_H2064                        17
P19_H65P                         17
P19_POPF                         17
P19_F1529                        17
P19_F3044                        17
P19_F4559                        17
P19_F6074                        17
P19_F7589                        17
P19_F90P                         17
P19_F0019                   

In [63]:
# Données manquentes pour Mayotte
all_content_2020.loc[all_content_2020.P19_POP.isna()]

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
34983,97601,Outre-Mer,06,976,Acoua,Mayotte,Mayotte,97608,Dzaoudzi,97630,...,23.0,1.0,5.0,11.0,0.0,0.0,0.0,4.0,1.0,1.0
34984,97602,Outre-Mer,06,976,Bandraboua,Mayotte,Mayotte,97608,Dzaoudzi,97650,...,61.0,1.0,6.0,41.0,1.0,0.0,0.0,3.0,8.0,1.0
34985,97603,Outre-Mer,06,976,Bandrele,Mayotte,Mayotte,97608,Dzaoudzi,97660,...,52.0,2.0,7.0,32.0,1.0,0.0,0.0,3.0,5.0,2.0
34986,97604,Outre-Mer,06,976,Bouéni,Mayotte,Mayotte,97608,Dzaoudzi,97620,...,42.0,1.0,6.0,21.0,0.0,1.0,0.0,4.0,6.0,3.0
34987,97605,Outre-Mer,06,976,Chiconi,Mayotte,Mayotte,97608,Dzaoudzi,97670,...,31.0,2.0,2.0,22.0,1.0,0.0,0.0,2.0,2.0,0.0
34988,97606,Outre-Mer,06,976,Chirongui,Mayotte,Mayotte,97608,Dzaoudzi,97620,...,61.0,3.0,4.0,40.0,0.0,0.0,1.0,4.0,8.0,1.0
34989,97607,Outre-Mer,06,976,Dembeni,Mayotte,Mayotte,97608,Dzaoudzi,97660,...,52.0,3.0,9.0,27.0,0.0,0.0,0.0,3.0,4.0,6.0
34990,97608,Outre-Mer,06,976,Dzaoudzi,Mayotte,Mayotte,97608,Dzaoudzi,97615,...,104.0,2.0,11.0,65.0,3.0,2.0,0.0,6.0,8.0,7.0
34991,97609,Outre-Mer,06,976,Kani-Kéli,Mayotte,Mayotte,97608,Dzaoudzi,97625,...,14.0,1.0,2.0,7.0,0.0,0.0,0.0,1.0,1.0,2.0
34992,97610,Outre-Mer,06,976,Koungou,Mayotte,Mayotte,97608,Dzaoudzi,97600,...,119.0,6.0,19.0,75.0,0.0,1.0,0.0,10.0,6.0,2.0


In [64]:
temp[temp==1]

ENCTOT19     1
ENCITOT19    1
ETCTOT19     1
ETCBE19      1
ETCFZ19      1
ETCGI19      1
ETCJZ19      1
ETCKZ19      1
ETCLZ19      1
ETCMN19      1
ETCOQ19      1
ETCRU19      1
dtype: int64

In [65]:
all_content_2020.loc[all_content_2020.ENCTOT19.isna()]

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
4884,14666,Nord-Ouest,28,14,Sannerville,Calvados,Normandie,76540,Rouen,14940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
temp[temp==0]

CODGEO           0
EU_circo         0
num_region       0
num_dep          0
nom_commune      0
nom_dep          0
nom_region       0
num_chef_lieu    0
nom_chef_lieu    0
code_postal      0
pop_tot          0
ENNTOT21         0
ENNBE21          0
ENNFZ21          0
ENNGI21          0
ENNJZ21          0
ENNKZ21          0
ENNLZ21          0
ENNMN21          0
ENNOQ21          0
ENNRU21          0
ENCTOT21         0
ENCBE21          0
ENCFZ21          0
ENCGI21          0
ENCJZ21          0
ENCKZ21          0
ENCLZ21          0
ENCMN21          0
ENCOQ21          0
ENCRU21          0
ENCTOT20         0
ENCITOT21        0
ENCIBE21         0
ENCIFZ21         0
ENCIGI21         0
ENCIJZ21         0
ENCIKZ21         0
ENCILZ21         0
ENCIMN21         0
ENCIOQ21         0
ENCIRU21         0
ENCITOT20        0
ETNTOT21         0
ETNBE21          0
ETNFZ21          0
ETNGI21          0
ETNJZ21          0
ETNKZ21          0
ETNLZ21          0
ETNMN21          0
ETNOQ21          0
ETNRU21     

In [67]:
# Verication des lignesdupliquées
all_content_2020.loc[all_content_2020.duplicated(subset='CODGEO')]

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19


In [68]:
# Sauvegarde de la DF all_content_2020 dans un fichier csv
all_content_2020.to_csv("all_content_2020_v4.csv",encoding= "utf-8")

In [72]:
# Lecture pour verification
all_content_test = pd.read_csv("all_content_2020_v4.csv",index_col=0)

In [75]:
# Affichage
all_content_test

,CODGEO,EU_circo,num_region,num_dep,nom_commune,nom_dep,nom_region,num_chef_lieu,nom_chef_lieu,code_postal,...,ETCTOT19,ETCBE19,ETCFZ19,ETCGI19,ETCJZ19,ETCKZ19,ETCLZ19,ETCMN19,ETCOQ19,ETCRU19
0,1001,Sud-Est,84,1,L'Abergement-Clémenciat,Ain,Auvergne-Rhône-Alpes,69123,Lyon,1400.0,...,9.0,0.0,1.0,2.0,0.0,1.0,0.0,3.0,1.0,1.0
1,1002,Sud-Est,84,1,L'Abergement-de-Varey,Ain,Auvergne-Rhône-Alpes,69123,Lyon,1640.0,...,5.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1004,Sud-Est,84,1,Ambérieu-en-Bugey,Ain,Auvergne-Rhône-Alpes,69123,Lyon,1500.0,...,178.0,11.0,27.0,41.0,9.0,5.0,10.0,27.0,27.0,21.0
3,1005,Sud-Est,84,1,Ambérieux-en-Dombes,Ain,Auvergne-Rhône-Alpes,69123,Lyon,1330.0,...,19.0,1.0,7.0,6.0,0.0,0.0,0.0,4.0,1.0,0.0
4,1006,Sud-Est,84,1,Ambléon,Ain,Auvergne-Rhône-Alpes,69123,Lyon,1300.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,97613,Outre-Mer,6,976,M'Tsangamouji,Mayotte,Mayotte,97608,Dzaoudzi,97650.0,...,18.0,1.0,4.0,9.0,0.0,0.0,0.0,1.0,3.0,0.0
34996,97614,Outre-Mer,6,976,Ouangani,Mayotte,Mayotte,97608,Dzaoudzi,97670.0,...,41.0,2.0,10.0,24.0,0.0,0.0,0.0,2.0,2.0,1.0
34997,97615,Outre-Mer,6,976,Pamandzi,Mayotte,Mayotte,97608,Dzaoudzi,97615.0,...,45.0,1.0,2.0,27.0,0.0,2.0,1.0,8.0,2.0,2.0
34998,97616,Outre-Mer,6,976,Sada,Mayotte,Mayotte,97608,Dzaoudzi,97640.0,...,59.0,5.0,4.0,34.0,3.0,2.0,1.0,4.0,5.0,1.0
